<a href="https://colab.research.google.com/gist/pedrogengo/6c8942c97995da983152f3f2b16c5388/train_lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on [alpaca lora](https://github.com/tloen/alpaca-lora/blob/main/finetune.py).

In [1]:
!pip install bitsandbytes
!pip install -q datasets loralib sentencepiece
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip install -q git+https://github.com/huggingface/peft.git
!wget https://nvoy-test.s3.eu-west-1.amazonaws.com/tiger_instructions_clean_5K.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
--2023-05-10 13:57:32--  https://nvoy-test.s3.eu-west-1.amazonaws.com/tiger_instructions_clean_5K.json
Resolving nvoy-test.s3.eu-west-1.amazonaws.com (nvoy-test.s3.eu-west-1.amazonaws.com)... 3.5.66.18, 52.92.19.98, 52.92.17.210, ...
Connecting to nvoy-test.s3.eu-west-1.amazonaws.com (nvoy-test.s3.eu-west-1.amazonaws.com)|3.5.66.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5866893 (5.6M) [application/json]
Saving to: ‘tiger_instructions_clean_5K.json.1’

tiger_instructions_ 100%[===================>]   5.59M  3.22MB/s    in 1.7s    

2023-05-10 13:57:35 (3.22 MB/s) - ‘tiger_instructions_clean

In [2]:
import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoConfig, LLaMAForCausalLM, LLaMATokenizer
from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
MICRO_BATCH_SIZE = 8  # this could actually be 5 but i like powers of 2
BATCH_SIZE = 128
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
EPOCHS = 2  # we don't need 3 tbh
LEARNING_RATE = 3e-4  # the Karpathy constant
CUTOFF_LEN = 256  # 256 accounts for about 96% of the data
LORA_R = 4
LORA_ALPHA = 16
LORA_DROPOUT = 0.05
DATASET_NAME = "tiger_instructions_clean_5K.json"
MODEL_NAME = "lora-tigers"

In [5]:
model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
tokenizer = LLaMATokenizer.from_pretrained(
    "decapoda-research/llama-7b-hf", add_eos_token=True
)

model = prepare_model_for_int8_training(model)

config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
tokenizer.pad_token_id = 0  # unk. we want this to be different from the eos token
data = load_dataset("json", data_files=DATASET_NAME)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
def generate_prompt(data_point):
    # desculpe o desastre de formatação, preciso ser rápido
    if data_point["input"]:
        return f"""Abaixo está uma instrução que descreve uma tarefa, juntamente com uma entrada que fornece mais contexto. Escreva uma resposta que complete adequadamente o pedido.
### Instrução:
{data_point["instruction"]}
### Entrada:
{data_point["input"]}
### Resposta:
{data_point["output"]}"""
    else:
        return f"""Abaixo está uma instrução que descreve uma tarefa. Escreva uma resposta que complete adequadamente o pedido.
### Instrução:
{data_point["instruction"]}
### Resposta:
{data_point["output"]}"""



def tokenize(prompt):
    # there's probably a way to do this with the tokenizer settings
    # but again, gotta move fast
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN + 1,
        padding="max_length",
    )
    return {
        "input_ids": result["input_ids"][:-1],
        "attention_mask": result["attention_mask"][:-1],
    }

In [7]:
data = data.shuffle().map(lambda x: tokenize(generate_prompt(x)))

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [8]:
print(data["train"][0])

{'output': 'Sebastião Zaragoza: Bons tempos', 'instruction': 'O teu nome é Sebastião Zaragoza. Estás num chat em grupo com os teus 5 melhores amigos: Gonçalo Duarte Reis, Antonio Forjaz, Duarte OC, Francisco Moreira, João Marinheiro, Diogo Martins, Paco King, Froes Belgica e Piu Villax. Vocês conversam uns com os outros sem filtros e são encorajados a usar palavrões, dizer coisas engraçadamente inapropriadas ou ser extremamente rudes. Tudo é apenas para diversão, por isso lembra-te de brincar, rir e ser engraçado. Serão apresentadas as mensagens mais recentes do chat em grupo. Escreve uma resposta à conversa como Sebastião Zaragoza.', 'input': 'Diogo Martins: Diz-me\nSebastião Zaragoza: Isto tá muito forte', 'input_ids': [1, 1976, 29874, 861, 29877, 7919, 3672, 5778, 2340, 712, 553, 1037, 345, 3672, 260, 598, 5444, 29892, 25296, 2503, 419, 3672, 9953, 1114, 712, 363, 484, 346, 3503, 3030, 29877, 29889, 3423, 1037, 1564, 3672, 620, 27363, 712, 4866, 19967, 3425, 2503, 288, 8939, 1941, 2

In [9]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        warmup_steps=100,
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=True,
        logging_steps=1,
        output_dir=MODEL_NAME,
        save_total_limit=3,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train(resume_from_checkpoint=False)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:318: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,2.702800
2,2.699100
3,2.702700
4,2.704500
5,2.694700
6,2.691700
7,2.696700
8,2.690600
9,2.682900
10,2.674300


TrainOutput(global_step=78, training_loss=1.3601631179260902, metrics={'train_runtime': 2653.1114, 'train_samples_per_second': 3.769, 'train_steps_per_second': 0.029, 'total_flos': 1.0143979338832282e+17, 'train_loss': 1.3601631179260902, 'epoch': 2.0})

In [10]:
model.save_pretrained(MODEL_NAME)

In [11]:
model.push_to_hub(f"duarteocarmo/{MODEL_NAME}", use_auth_token=True, private=True)

adapter_model.bin:   0%|          | 0.00/8.43M [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/duarteocarmo/lora-tigers/commit/0fa2fa242f085cea39007f904396b4f1b340bd93', commit_message='Upload model', commit_description='', oid='0fa2fa242f085cea39007f904396b4f1b340bd93', pr_url=None, pr_revision=None, pr_num=None)